In [151]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\hp\AppData\Local\Temp\ipykernel_23728\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [152]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [153]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
# Handle missing values
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [154]:
# Your code
from sklearn.model_selection import train_test_split

# Split the data into training and test sets (80% train, 20% test)
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

# Ensure 'preprocessed_text' exists (modify this to match your actual column name)
data_train["preprocessed_text"] = data_train["text"].astype(str)
data_val["preprocessed_text"] = data_val["text"].astype(str)

## Data Preprocessing

In [155]:
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
print(string.punctuation)
print(stopwords.words("english")[100:110])
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [156]:
# Your code
def remove_js_css(text):
    # Remove inline JavaScript and CSS (everything inside <script> and <style> tags)
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    return text
def remove_html_comments(text):
    # Remove HTML comments (e.g., <!-- comment -->)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    return text
def remove_html_tags(text):
    # Remove remaining HTML tags
    text = re.sub(r'<.*?>', '', text)
    return text
def clean_html(text):
    # Remove inline JavaScript and CSS
    text = remove_js_css(text)
    # Remove HTML comments
    text = remove_html_comments(text)
    # Remove HTML tags
    text = remove_html_tags(text)
    return text
train_data["clean_text"] = train_data["text"].apply(clean_html)
test_data["clean_text"] = test_data["text"].apply(clean_html)

print(train_data.head())  # View the cleaned data

                                                  text  label  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...      1   
535  I have not been able to reach oscar this am. W...      0   
695  ; Huma Abedin B6I'm checking with Pat on the 5...      0   
557  I can have it announced here on Monday - can't...      0   
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...      1   

                                            clean_text  
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...  
535  I have not been able to reach oscar this am. W...  
695  ; Huma Abedin B6I'm checking with Pat on the 5...  
557  I can have it announced here on Monday - can't...  
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...  


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [157]:
import re

def clean_text(text):
    # Remove all special characters (except spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^\w', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove prefixed 'b' (if it appears at the beginning of words)
    text = re.sub(r'\bb', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text.strip()  # Remove leading and trailing spaces
# Apply the cleaning function to the dataset
train_data["clean_text"] = train_data["text"].apply(clean_text)
test_data["clean_text"] = test_data["text"].apply(clean_text)

# Display the first few rows
print(train_data.head())

                                                  text  label  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...      1   
535  I have not been able to reach oscar this am. W...      0   
695  ; Huma Abedin B6I'm checking with Pat on the 5...      0   
557  I can have it announced here on Monday - can't...      0   
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...      1   

                                            clean_text  
29   regards mr nelson smithkindly reply me on my p...  
535  have not een able to reach oscar this am we ar...  
695  huma abedin bim checking with pat on the will ...  
557    can have it announced here on monday cant today  
836  bank of africaagence san pedro bp san pedro co...  


## Now let's work on removing stopwords
Remove the stopwords.

In [158]:
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already downloaded
nltk.download('stopwords')

# Get the list of English stopwords
stop_words = set(stopwords.words("english"))
def remove_stopwords(text):
    words = text.split()  # Tokenize text into words
    filtered_words = [word for word in words if word not in stop_words]  # Remove stopwords
    return " ".join(filtered_words)  # Join the words back into a string
# Apply the function to the cleaned text column
train_data["clean_text"] = train_data["clean_text"].apply(remove_stopwords)
test_data["clean_text"] = test_data["clean_text"].apply(remove_stopwords)

# Display the cleaned text
print(train_data.head())


                                                  text  label  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...      1   
535  I have not been able to reach oscar this am. W...      0   
695  ; Huma Abedin B6I'm checking with Pat on the 5...      0   
557  I can have it announced here on Monday - can't...      0   
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...      1   

                                            clean_text  
29   regards mr nelson smithkindly reply private em...  
535     een able reach oscar supposed send pdb receive  
695  huma abedin bim checking pat work jack jake re...  
557                        announced monday cant today  
836  bank africaagence san pedro bp san pedro cote ...  


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [159]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download necessary NLTK resources
nltk.download('punkt')  # For tokenization
nltk.download('wordnet')  # For lemmatization
nltk.download('omw-1.4')  # For better word understanding

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    words = word_tokenize(text)  # Tokenize text into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Apply lemmatization
    return " ".join(lemmatized_words)  # Join words back into a sentence
# Apply the function to the cleaned text column
train_data["clean_text"] = train_data["clean_text"].apply(lemmatize_text)
test_data["clean_text"] = test_data["clean_text"].apply(lemmatize_text)

# Display the cleaned text
print(train_data.head())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                                  text  label  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...      1   
535  I have not been able to reach oscar this am. W...      0   
695  ; Huma Abedin B6I'm checking with Pat on the 5...      0   
557  I can have it announced here on Monday - can't...      0   
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...      1   

                                            clean_text  
29   regard mr nelson smithkindly reply private ema...  
535     een able reach oscar supposed send pdb receive  
695  huma abedin bim checking pat work jack jake re...  
557                        announced monday cant today  
836  bank africaagence san pedro bp san pedro cote ...  


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [160]:
# Your code
from collections import Counter

# Separate spam and ham messages
spam_messages = train_data[train_data["label"] == "spam"]["clean_text"]
ham_messages = train_data[train_data["label"] == "ham"]["clean_text"]
def get_top_words(text_series, n=10):
    all_words = " ".join(text_series).split()  # Combine all text and split into words
    word_counts = Counter(all_words)  # Count word frequencies
    return word_counts.most_common(n)  # Return top N words
top_spam_words = get_top_words(spam_messages, 10)
top_ham_words = get_top_words(ham_messages, 10)

print("Top 10 Spam Words:", top_spam_words)
print("Top 10 Ham Words:", top_ham_words)


Top 10 Spam Words: []
Top 10 Ham Words: []


## Extra features

In [161]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,0,106
535,I have not been able to reach oscar this am. W...,0,I have not been able to reach oscar this am. W...,1,0,101
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,; Huma Abedin B6I'm checking with Pat on the 5...,1,0,141
557,I can have it announced here on Monday - can't...,0,I can have it announced here on Monday - can't...,1,0,52
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,1,1750


## How would work the Bag of Words with Count Vectorizer concept?

In [162]:
from sklearn.feature_extraction.text import CountVectorizer

# Sample dataset (use your actual dataset instead)
text_samples = [
    "Free money now! Click here to win",
    "Hello, how are you?",
    "Exclusive offer! Get a discount now",
    "Let's meet tomorrow for coffee",
    "Congratulations, you won a prize!"
]

# Initialize CountVectorizer
vectorizer = CountVectorizer()
# Fit and transform the text data
X = vectorizer.fit_transform(text_samples)

# Convert sparse matrix to array for readability
X_array = X.toarray()
print(X_array)
# Print feature names (unique words in corpus)
print("Vocabulary:", vectorizer.get_feature_names_out())
# Assuming `data_train['preprocessed_text']` contains cleaned text
X_train = vectorizer.fit_transform(data_train["preprocessed_text"])
X_val = vectorizer.transform(data_val["preprocessed_text"])

print("Shape of Train Data:", X_train.shape)
print("Shape of Validation Data:", X_val.shape)


[[0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1]]
Vocabulary: ['are' 'click' 'coffee' 'congratulations' 'discount' 'exclusive' 'for'
 'free' 'get' 'hello' 'here' 'how' 'let' 'meet' 'money' 'now' 'offer'
 'prize' 'to' 'tomorrow' 'win' 'won' 'you']
Shape of Train Data: (800, 23374)
Shape of Validation Data: (200, 23374)


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [163]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(data_train["preprocessed_text"])

# Transform the validation data (do NOT fit again)
X_val_tfidf = tfidf_vectorizer.transform(data_val["preprocessed_text"])
print("Shape of TF-IDF Train Data:", X_train_tfidf.shape)
print("Shape of TF-IDF Validation Data:", X_val_tfidf.shape)


Shape of TF-IDF Train Data: (800, 23374)
Shape of TF-IDF Validation Data: (200, 23374)


## And the Train a Classifier?

In [164]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Define the model
classifier = LogisticRegression()
# Train the model using the TF-IDF features
classifier.fit(X_train_tfidf, data_train["label"])  # Assuming "label" column contains spam (1) and ham (0)
# Predict on validation set
y_pred = classifier.predict(X_val_tfidf)
# Calculate accuracy
accuracy = accuracy_score(data_val["label"], y_pred)
print("Accuracy:", accuracy)

# Print classification report
print(classification_report(data_val["label"], y_pred))


Accuracy: 0.95
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       125
           1       0.97      0.89      0.93        75

    accuracy                           0.95       200
   macro avg       0.95      0.94      0.95       200
weighted avg       0.95      0.95      0.95       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).